In [1]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.utils import image_dataset_from_directory

2025-06-21 20:34:29.229281: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750538069.455959      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750538069.518595      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
TRAIN_DIR = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
VAL_DIR = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid"
IMAGE_SIZE=(224,224)
BATCH_SIZE=16

# Normalize images
def preprocess(img, label):
    img = tf.cast(img, tf.float32) / 255.0
    return img, label
    
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=True
).map(preprocess).repeat().prefetch(tf.data.AUTOTUNE)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_DIR,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=False
).map(preprocess).prefetch(tf.data.AUTOTUNE)

Found 70295 files belonging to 38 classes.


I0000 00:00:1750538128.288710      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Found 17572 files belonging to 38 classes.


In [3]:
train_samples=70295
val_samples=17572

In [4]:
def Conv_Block(x,output_channels):
    res=x
    
    x=Conv2D(output_channels//4,1,strides=(1,1),padding='valid')(x)
    x=BatchNormalization()(x)
    x=ReLU()(x)
    
    x=Conv2D(output_channels//4,3,strides=(1,1),padding='same')(x)
    x=BatchNormalization()(x)
    x=ReLU()(x)
    
    x=Conv2D(output_channels,1,strides=(1,1),padding='valid')(x)
    x=BatchNormalization()(x)

    #Shortcut projection
    res=Conv2D(output_channels,1,strides=(1,1),padding='valid')(res)
    res=BatchNormalization()(res)
    
    x=Add()([x,res])
    x=ReLU()(x)
    return x

In [5]:
def Identity_Block(x,output_channels):
    res=x
    
    x=Conv2D(output_channels//4,1,strides=(1,1),padding='valid')(x)
    x=BatchNormalization()(x)
    x=ReLU()(x)
    
    x=Conv2D(output_channels//4,3,strides=(1,1),padding='same')(x)
    x=BatchNormalization()(x)
    x=ReLU()(x)
    
    x=Conv2D(output_channels,1,strides=(1,1),padding='valid')(x)
    x=BatchNormalization()(x)

    x=Add()([x,res])  # no need os shortcut projection since identity block we used conv_block with same no. of output channels
    x=ReLU()(x)
    return x

In [6]:
def Build_ResNet50(input_shape=(224,224,3), num_classes=38):
    input_=Input(shape=input_shape)
    x=Conv2D(64, kernel_size=7, strides=2, padding='same')(input_)
    x=BatchNormalization()(x)
    x=ReLU()(x)
    x=MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding="same")(x)
    
    x=Conv_Block(x,256)
    x=Identity_Block(x,256)
    x=Identity_Block(x,256)
    
    x=Conv_Block(x,512)
    x=Identity_Block(x,512)
    x=Identity_Block(x,512)
    x=Identity_Block(x,512)
    
    x=Conv_Block(x,1024)
    x=Identity_Block(x,1024)
    x=Identity_Block(x,1024)
    x=Identity_Block(x,1024)
    x=Identity_Block(x,1024)
    x=Identity_Block(x,1024)
    
    x=Conv_Block(x,2048)
    x=Identity_Block(x,2048)
    x=Identity_Block(x,2048)

    x = GlobalAveragePooling2D()(x)
    x = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=input_,outputs=x)

In [7]:
resnet50=Build_ResNet50()
resnet50.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 112, 112, 64)   │          9,472 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 112, 112, 64)   │            256 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu (ReLU)              │ (None, 112, 112, 64)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 56, 56, 64)     │              0 │ re_lu[0][0]            │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 56, 56, 64)     │          4,160 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 56, 56, 64)     │            256 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_1 (ReLU)            │ (None, 56, 56, 64)     │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 56, 56, 64)     │         36,928 │ re_lu_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 56, 56, 64)     │            256 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_2 (ReLU)            │ (None, 56, 56, 64)     │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 56, 56, 256)    │         16,640 │ re_lu_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 56, 56, 256)    │         16,640 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 56, 56, 256)    │          1,024 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 56, 56, 256)    │          1,024 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 56, 56, 256)    │              0 │ batch_normalization_3… │
│                           │                        │                │ batch_normalization_4… │
├──────────────────────

 Total params: 23,665,574 (90.28 MB)

 Trainable params: 23,612,454 (90.07 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [8]:
resnet50.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',   # or 'sparse_categorical_crossentropy' if labels are integers
        metrics=['accuracy']
    )

In [9]:
steps_per_epoch = train_samples // BATCH_SIZE
validation_steps = val_samples // BATCH_SIZE

history = resnet50.fit(
        train_ds,
        epochs=10,
        batch_size=16,
        validation_data=val_ds,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
            tf.keras.callbacks.ModelCheckpoint("best_resnet50.h5", save_best_only=True)
        ]
    )

Epoch 1/10


I0000 00:00:1750538180.868017      64 service.cc:148] XLA service 0x7cbbe8015240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750538180.868872      64 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1750538185.304803      64 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1750538219.709758      64 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4393/4393 ━━━━━━━━━━━━━━━━━━━━ 3788s 843ms/step - accuracy: 0.6196 - loss: 1.3054 - val_accuracy: 0.6481 - val_loss: 1.4375
Epoch 2/10
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 3729s 843ms/step - accuracy: 0.8976 - loss: 0.3171 - val_accuracy: 0.8648 - val_loss: 0.4812
Epoch 3/10
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 3703s 843ms/step - accuracy: 0.9346 - loss: 0.2047 - val_accuracy: 0.7992 - val_loss: 0.7366
Epoch 4/10
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 3705s 843ms/step - accuracy: 0.9532 - loss: 0.1450 - val_accuracy: 0.8677 - val_loss: 0.4395
Epoch 5/10
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 3705s 843ms/step - accuracy: 0.9623 - loss: 0.1156 - val_accuracy: 0.9078 - val_loss: 0.3108
Epoch 6/10
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 3705s 843ms/step - accuracy: 0.9694 - loss: 0.0954 - val_accuracy: 0.9581 - val_loss: 0.1322
Epoch 7/10
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 3705s 843ms/step - accuracy: 0.9757 - loss: 0.0771 - val_accuracy: 0.9492 - val_loss: 0.1569
Epoch 8/10
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 3705s 843ms/step - accur